#  American Option Pricing for Multiple Underlying assets using Random Forest


let us first import all the packages 

In [2]:
%pylab inline --no-import-all
import numpy as np
import scipy as sp
import pylab as pl
import warnings 
import matplotlib.pyplot as plt
import statistics as st
from scipy.optimize import leastsq
import pandas as pd
from scipy.stats import norm

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

Populating the interactive namespace from numpy and matplotlib


In [3]:
def generate_paths(S0, dt , sigma, r, N, m,q):
    S = np.zeros((m + 1,N))
    S[0] = S0
    for t in range(1, m + 1):
        rand = np.random.standard_normal(N)
        S[t] = S[t - 1] * np.exp((r -q- sigma ** 2 / 2) * dt + sigma * dt ** 0.5 * rand)
    return S 

In [4]:
def generate_multiple_asset(p, S0, dt , sigma, r, N, m,q):
    S = []
    #Generate p assets
    for __ in range (N): 
        S_asset=generate_paths(S0, dt , sigma, r, p, m,q)
        S.append(S_asset)
    return (S)

In [5]:
def exercise_matrix(S,p,N,K,m): 
    payoff=np.zeros((m + 1, N))
    for j in range (N): 
        for i in range (m + 1):
            pay_int = np.prod(S[j][i,:])**(1/p)
            pay=max(pay_int-K,0)
            payoff[i,j]=pay
    return (payoff)

In [6]:
def pricing_option(K,r,sigma,N,m,S_0,T, RF_n_estimators,RF_max_leaf_nodes,p,q):
  
    "Time-step"
    dt=T/m
    
    "Discount factor"
    df=np.exp(-r*dt)
    "N simulations of p underlying assets "
    S = generate_multiple_asset(p, S_0, dt , sigma, r, N, m,q)
    
    "Matrix of exercise prices"
    h = exercise_matrix(S,p,N,K,m) 
    
    
    "price of the option at time T = Initialization "
    V=h[-1]
    
    "Iteration over the paths"
    for t in range (m-2,0,-1): 
        
        "Creation of the matrix N*p to regress"
        X=np.zeros([N,p])
        for i in range (N):
            for j in range (p): 
                X[i,j]=S[i][t , j - 1]
        
       
        "Random Forest algorithm uses the j'th time of the Stock and the (j+1)'th time of Option Price to fit "
        rf=RandomForestRegressor(n_estimators=RF_n_estimators,
                                max_leaf_nodes=RF_max_leaf_nodes,oob_score=False, n_jobs=-1)
        rf.fit(X,V)
        B=rf.predict(X)
        V=np.where(h[t] > B, h[t],df*V)
    
      
    "Monte-Carlo for the option pricing"
    V_opt= df*np.sum(V)/N
    
    
    return (V_opt)

Example of pricing with 7 assets

In [7]:
def black_scholes(K,r,sigma, S_0,T,p,q): 
    d=sigma**2*(1-1/p)/2+q
    d1=1/(sigma*np.sqrt(T/p))*(np.log(S_0/K)+(r+0.5*sigma*sigma/p-d)*T)
    d2=d1-sigma*np.sqrt(T/p)
    dscnt=np.exp(-r*T)
    call=-norm.cdf(d2)*K*dscnt+norm.cdf(d1)*S_0*np.exp(-d*T)
    return (call)

In [8]:

"Parameters"
T=1
m=10
K=100
S_0=100
r=0.03
q=0.05
sigma=0.4
N=10000
RF_n_estimators=150
RF_max_leaf_nodes=20
p=7

price = pricing_option(K,r,sigma,N,m,S_0,T,RF_n_estimators,RF_max_leaf_nodes,p,q)
print ("price", price)

price_BS=black_scholes(K,r,sigma, S_0,T,p,q)
print ("price Black-Scholes ", price_BS) 

print ("Expected value from binomial tree : ", 3.27)

price 3.17905715186
price Black-Scholes  2.41878440914
Expected value from binomial tree :  3.27
